# Générer une édition XML TEI à partir des transcriptions des répertoires de notaires de Paris

## Contexte
L'application [eScriptorium](http://escriptorium.inria.fr/) permet de générer la transcription, automatique ou manuelle, des numérisations des pages des répertoires de notaires parisiens. On en exporte une série de fichiers XML PAGE.

Le présent notebook vise à implémenter un scénario de transformation pour transformer les fichiers XML PAGE en fichier XML TEI rendant compte de la structure logique des pages des répertoires. Ces fichiers XML TEI peuvent ensuite alimenter une application TEI Publisher.



## Prérequis dans eScriptorium

### Générer une archive à télécharger

> *Pour créer un lien vers un export :* 
> - *se connecter à eScriptorium*
> - *sélectionner les pages à exporter ainsi que la bonne version de transcription et le format souhaité*
> - *copier le lien de téléchargement généré par l'application : ce lien est public*
> - *il est également possible de se rendre dans "http://traces6.paris.inria.fr/profile/files/" pour retrouver un ancien lien.*


### Annoter les régions dans les documents
On a prélablement créer des zones de 7 types dans eScriptorium, en leur associant les segments de texte correspondants :
- `col_1` pour la colonne "numéros du répertoire"
- `col_2` pour la colonne "date de l'acte"
- `col_3` pour la colonne "Nature et espèce des actes : / en brevets"
- `col_4` pour la colonne "Nature et espèce des actes : / en minutes"
- `col_5` pour la colonne "Noms, prénoms et domiciles des parties / indications, situations et prix des biens"
- `col_6` pour la colonne "Relation de l'enregistrement / dates"
- `col_7` pour la colonne "Relation de l'enregistrement / droits"


### Annoter les first_line

On a préalablement assigné à toutes les lignes de la colonne centrale marquant le début d'un paragraphe le type `first_line`. En conséquence, les noeuds `//TextLine` annotés ainsi contiennent un attribut supplémentaire : `custom="structure {type:first_line;}"` :

``` xml
      <TextLine id="eSc_line_73523fc1" custom="structure {type:first_line;}">
        <Coords points="1300,2741 1304,2712 1337,2687 1406,2712 1487,2698 1652,2716 1677,2698 1703,2709 1751,2661 1780,2665 1831,2716 1926,2719 1981,2665 2432,2661 2443,2756 2410,2782 2373,2767 2245,2782 2128,2782 2077,2763 1882,2771 1816,2807 1740,2774 1674,2789 1633,2763 1586,2782 1545,2760 1498,2778 1425,2763 1304,2771"/>
        <Baseline points="1304,2745 1461,2752 1498,2760 2446,2760"/>
        <TextEquiv>
          <Unicode>Boudier (par Abel Eugène) architecte &amp;amp; Marie Thérèse Louise</Unicode>
        </TextEquiv>
      </TextLine>

      <TextLine id="eSc_line_d7e2b970" >
        <Coords points="1249,2811 1256,2763 1322,2785 1428,2763 1483,2789 1538,2767 1586,2789 1626,2767 1666,2789 1732,2771 1754,2789 1842,2789 1882,2771 1948,2785 1978,2771 2113,2771 2135,2793 2183,2771 2238,2793 2267,2771 2384,2771 2406,2793 2443,2793 2450,2826 2439,2848 2003,2848 1945,2833 1523,2844 1414,2829 1252,2840"/>
        <Baseline points="1250,2815 1527,2815 1589,2822 1886,2826 2454,2828"/>
        <TextEquiv>
          <Unicode>Jenny Belin safe- à Paris 15 rue Picot à Abel à Paris 24 BdSt Denis</Unicode>
        </TextEquiv>
      </TextLine>
```

### Annoter les main_date

On a préalablement assigné à toutes les premières lignes du documents, qui indiquent le mois et l'année des actes du répertoire, un type `main_date`. En conséquence les noeuds `//TextLine` annotés ainsi contiennent un attribut supplémentaire : `custom="structure {type:main_date;}"`. Le plus souvent, il n'y en a qu'un par page, mais il peut y en avoir 2 en cas de changement de mois ou d'année en cours de page.

``` xml
      <TextLine id="eSc_line_48c26889" custom="structure {type:main_date;}">
        <Coords points="1246,678 1253,638 1282,624 1359,642 1402,627 1417,642 1511,620 1573,642 1646,627 1675,642 1700,624 1722,642 1824,631 1838,646 1900,646 1922,624 1969,624 1991,646 2107,646 2133,624 2198,624 2231,646 2271,627 2278,675 2267,700 2180,700 2144,718 2104,700 2027,700 2009,718 1915,700 1849,700 1816,718 1737,700 1598,715 1551,696 1489,711 1457,696 1439,715 1250,711"/>
        <Baseline points="1250,682 2122,689 2158,682 2281,678"/>
        <TextEquiv>
          <Unicode>An 1912, mois de Novembre</Unicode>
        </TextEquiv>
      </TextLine>
```
---

## 0\. Installation de l'environnement

- import des dépendances
- création des répertoires
- téléchargement de saxon et de la feuille XSL de transcription de page vers tei
- (opt) vidange des dossiers

### 0.1 Installation des dépendances

In [1]:
import os
import re

from bs4 import BeautifulSoup, NavigableString
import itertools

### 0.2 Organisation du répertoire de travail

Les dossiers créés sont les suivantes : 
- `/content/source/` -> reçoit les fichiers XML PAGE
- `/content/tei_output/` -> reçoit le résultat de la transformation XSL
- `/content/tei4publisher/` -> reçoit les fichiers XML TEI prêts à être chargés dans TEI Publisher

In [2]:
# création des dossiers
# if running locally
!mkdir content
!mkdir content/source
!mkdir content/tei4publisher
!mkdir content/tei_output

# if running on collab
#!mkdir source
#!mkdir tei4publisher
#!mkdir tei_output

Récupération de la feuille XSL pour modifier les fichiers XML PAGE vers TEI

> le scenario est une adaptation de la feuille de transformation de [Manon Ovide](https://raw.githubusercontent.com/inoblivionem/xslt-playground/main/xmlpage_to_tei/xmlpage_to_tei.xsl).

In [22]:
# on utilise la version corrigée par Hugo
url_xsl = "https://raw.githubusercontent.com/HugoSchtr/xslt-playground/modif_xmlpagetotei_xsl/xmlpage_to_tei/xmlpage_to_tei.xsl"

# if running locally
path_to_xsl = os.path.join(os.path.abspath("."),(os.path.basename(url_xsl)))
# if executing on colab:
#path_to_xsl = os.path.join("content", os.path.basename(url_xsl))

!wget -N $url_xsl

--2021-09-17 00:43:31--  https://raw.githubusercontent.com/HugoSchtr/xslt-playground/modif_xmlpagetotei_xsl/xmlpage_to_tei/xmlpage_to_tei.xsl
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connexion vers raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connecté.
requête HTTP transmise, en attente de la réponse... 200 OK
Taille : 4793 (4,7K) [text/plain]
Enregistre : «xmlpage_to_tei.xsl»

xmlpage_to_tei.xsl  100%[===================>]   4,68K  --.-KB/s    ds 0s      

En-tête de dernière modification manquant -- horodatage ignoré.
2021-09-17 00:43:31 (65,5 MB/s) - «xmlpage_to_tei.xsl» enregistré [4793/4793]



Récupération du parser Saxon HE 9.9.1-7 pour appliquer la feuille XSL

In [19]:
# récupération du parser Saxon
url_saxon = "https://repo1.maven.org/maven2/net/sf/saxon/Saxon-HE/9.9.1-7/Saxon-HE-9.9.1-7.jar"

# if running locally
path_to_saxon = os.path.join(os.path.abspath("."),(os.path.basename(url_saxon)))
# if executing on colab:
#path_to_saxon = os.path.join("/content", os.path.basename(url_saxon))

!wget -N $url_saxon

--2021-09-17 00:41:54--  https://repo1.maven.org/maven2/net/sf/saxon/Saxon-HE/9.9.1-7/Saxon-HE-9.9.1-7.jar
Résolution de repo1.maven.org (repo1.maven.org)... 151.101.120.209
Connexion vers repo1.maven.org (repo1.maven.org)|151.101.120.209|:443... connecté.
requête HTTP transmise, en attente de la réponse... 304 Not Modified
Le fichier «Saxon-HE-9.9.1-7.jar» n’a pas été modifié sur le serveur. Téléchargement sauté.



### 0.3 [opt] Vidange des dossiers

Comme Google Colab ne permet pas de supprimer plusieurs fichiers à la fois, on peut utiliser le bloc suivant pour "vider" les dossiers sans avoir à réinitialiser l'environnement.

In [5]:
#réinitialisation de source/
#!rm -r content/source/

#réinitialisation de tei_output/
#!rm -r content/tei_output/

#réinitialisation de tei4publisher/
#!rm -r content/tei4publisher/

### 0.4 [opt] Récupération du fichier externe de constantes

Etape nécessaire pour une exécution sur Colab.

In [6]:
# TODO: tester le chemin sur colab
#!wget https://gitlab.inria.fr/almanach/lectaurep/lepidemo/-/raw/master/constants.py

In [7]:
from constants import MONTHS_MAPS, TEIHEADER

## 1\. Fonctions et classes

### 1.1 Input/Output

- contrôle léger sur les fichiers : on vérifier qu'on travaille bien avec des fichiers TEI ou PAGE en cherchant les éléments root attendus, respectivement `TEI` et `PcGtS`. On pourrait aller plus loin et regarder la déclaration du schéma.

- ouverture d'un fichier XML et parsing avec BeautifulSoup

- création/ouverture d'un fichier XML et enregistrement du contenu

- (opt) modification légère du fichier XSL

In [8]:
def control_schema_validity(xml_tree):
    """Control the existence of an accepted root element in XML tree"""
    ACCEPTED_SCHEMAS = ["PcGts", "TEI"]
    for schema in ACCEPTED_SCHEMAS:
        if len(xml_tree.find_all(schema)) == 1:
            return True
    print("Aucun schéma valide trouvé dans le fichier XML !")


def open_and_parse_file(path):
    """Open a file, parse with BS and return result"""
    with open(path, 'r', encoding='utf8') as fh:
        content = fh.read()
    parsed = BeautifulSoup(content, 'xml')
    if not control_schema_validity(parsed):
        parsed = None
    return parsed


def save_file(path, content):
    """Create or open a file and modify content"""
    try:
        with open(path, 'w', encoding="utf8") as fh:
            fh.write(content)
    except Error as e:
        e    


def correct_xsl(path_to_xsl):
    with open(path_to_xsl, "r", encoding='utf-8') as fh:
        xsl = fh.read()
    xsl = xsl.replace("""<?xml version="1.0" encoding="UTF-8"?>""", """<?xml version="1.0" encoding="UTF-8" standalone="yes"?>""")
    # we don't like how the output file is named:
    xsl = xsl.replace('<xsl:result-document href="teifromxmlpage.xml"', f'<xsl:result-document href="{tei_file}"')
    with open(path_to_xsl, "w", encoding="utf-8") as fh:
        fh.write(xsl)

### 1.2 Analyse de PAGE

On cherche les lignes annotées comme "first_line" et à partir de leurs coordonnées, on génère un découpage vertical du fichier permettant d'identifier les unités logiques.

***Attention :*** Pour le moment, on perd les informations situées avant le premier élément logique

In [9]:
def vertical_slicing(xml_tree):
    """Compose the vertical slices in an image depending on the coordinates of the  
    //TextLine/@custom="structure {type:col_5;}" nodes."""
    # Targetting the central column (col_5)
    main_cols = xml_tree.find_all("TextRegion", custom="structure {type:col_5;}")
    head_lines = []
    for main_col in main_cols:
        first_lines = main_col.find_all("TextLine", custom="structure {type:first_line;}")
    for fline in first_lines:
        coords = fline.find_all("Coords")
        points = coords[0].attrs.get("points", f"le noeud {fline.attrs['id']} a des coords incomplètes (pas de @points)")
        y_coords = [int(xy.split(",")[-1]) for xy in points.split(" ")]
        min_y = sorted(y_coords)[0]
        head_lines.append({"head_line": fline.attrs["id"], "top_max": min_y})

    for line in head_lines:
        if head_lines.index(line) + 1 < len(head_lines):
            max_y = head_lines[head_lines.index(line) + 1]["top_max"] 
        else:
            max_y = None
        line["bottom_max"] = max_y
    return head_lines


# TODO : récupérer les éléments date, les en-tête et les éléments marginaux du haut de la page

### 1.3 Génération des entrées de tableaux (Row)

L'objet  `Row` permet de modéliser rapidement les unitées logiques des tableaux à partir du découpage vertical généré grâce à l'analyse de la PAGE et des éléments "first_line".

In [10]:
class Row:
    def show_row(self):
        shown = self.__dict__
        del shown["_xml_tree"]
        pp = pprint.PrettyPrinter(indent=2)
        pp.pprint(shown)
        return shown

    def _find_text(self, node_id):
        return self._xml_tree.find(True, id=node_id).text.strip()

    def show_text_in_row(self):
        main_paragraph = "\n+ ".join([self._find_text(nid) for nid in self.main_paragraph])
        print(
            "\n".join([f"{self.top_limit} < {self.bottom_limit}",
                       f"num de répertoire : {[self._find_text(nid) for nid in self.entry_id]}",
                       f"date de l'acte : {[self._find_text(nid) for nid in self.date_of_act]}",
                       f"types de l'acte (brevet) : {[self._find_text(nid) for nid in self.type_of_act['brevet']]}",
                       f"types de l'acte (minute) : {[self._find_text(nid) for nid in self.type_of_act['minute']]}",
                       f"paragraphe central : {main_paragraph}",
                       f"date d'enregistrement : {[self._find_text(nid) for nid in self.registration_relation['date']]}",
                       f"droits d'enregistrement : {[self._find_text(nid) for nid in self.registration_relation['droits']]}",
                       f"misc : {[self._find_text(nid) for nid in self.misc]}",
                       "___fin___"]) 
        )

    def _elems_in_range(self):
        selected_elems = []
        textline_nodes = self._xml_tree.find_all("TextLine")
        for textline in textline_nodes:
            # On récupère les coordonnées de la ligne pour savoir si elle rentre
            # dans le range défini pour l'entrée en cours.
            baseline = textline.find("Baseline")
            points = baseline.attrs.get("points", f"le noeud {textline.attrs['id']} a des coords incomplètes (pas de @points)")
            y_coords = [int(xy.split(",")[-1]) for xy in points.split(" ")]
            highest_point = sorted(y_coords)[0]
            # TODO : rendre + pythonesque
            if not self.bottom_limit:
                if self.top_limit < highest_point:
                    selected_elems.append(textline)
            else:
                if self.top_limit < highest_point <= self.bottom_limit:
                    selected_elems.append(textline)
        return selected_elems

    def _distribute_lines(self, lines):
        for line in lines:
            # custom="structure {type:col_3;}" -> "col_3"
            if "custom" in line.parent.attrs.keys():
                region_type = line.parent.attrs["custom"].replace("structure {type:", "").replace(";}", "")
                if region_type == "col_1":
                    self.entry_id.append(line.attrs["id"])
                elif region_type == "col_2":
                    self.date_of_act.append(line.attrs["id"])
                elif region_type == "col_3":
                    self.type_of_act["brevet"].append(line.attrs["id"])
                elif region_type == "col_4":
                    self.type_of_act["minute"].append(line.attrs["id"])
                elif region_type == "col_5":
                    # TODO: et si l'id de la line n'est pas head_line...
                    self.main_paragraph.append(line.attrs["id"])
                elif region_type == "col_6":
                    self.registration_relation["date"].append(line.attrs["id"])
                elif region_type == "col_7":
                    self.registration_relation["droits"].append(line.attrs["id"])
                else:
                    self.misc.append(line.attrs["id"])
            else:
                self.misc.append(line.attrs["id"])

    def __init__(self, xml_tree, part):
        self._xml_tree = xml_tree
        self.top_limit = part["top_max"]
        self.bottom_limit = part["bottom_max"]
        self.head_line = part["head_line"]
        self.main_paragraph = []
        self.entry_id = []
        self.date_of_act = []
        self.type_of_act = {"minute" : [], "brevet": []} # warrant?
        self.registration_relation = {"date" : [] , "droits" : []}
        self.misc = []

        associated_lines = self._elems_in_range()
        self._distribute_lines(associated_lines)

### 1.4 Modification des TEI

La modification des fichiers TEI consiste à :
- mettre à jour les métadonnées dans le teiHeader (***à améliorer***)
- ajouter une section "text" rendant compte de la structure logique telle qu'elle est modélisée grâce aux éléments Row.

#### 1.4.1 Fonctions de contrôle des dates

In [11]:
def is_date(value):
    """Evaluate how likely a string is to refer to a date"""
    value = str(value)
    if '(' in value or ')' in value or '&amp;' in value:
        value = value.replace("(", "").replace(")", "").replace("&amp;", " ").strip()
    if value and value.isdigit():
        if 1 <= int(value) <= 31:
            return "high"
    if value.strip() in ['"', "d°", "-", "- -"]:
        return "unknown"
    for month in MONTHS_MAPS.keys():
        if month in value.lower():
            return "medium"
    return "low"


def control_dates(tree):
    """Parse date elements in TEI tree and add a cert attribute or delete when-iso"""
    #slices = slice_dates(tree)
    for date in tree.body.find_all("date"):
        date.attrs["cert"] = is_date(date.string)
        if not date.attrs["when-iso"].isdigit():
            del date.attrs["when-iso"]

# ------


def compose_iso_date(date_node, yyyy_mm):
    if date_node.attrs["when-iso"]:
        if len(date_node.attrs["when-iso"].split("-")) == 1:
            iso_date = f'{yyyy_mm}-{date_node.attrs["when-iso"]}'
    return iso_date
 

def build_list_of_years_and_months(main_dates_nodes):
    years = []
    months = []
    for textline in main_dates_nodes:
        line = str(textline.TextEquiv.Unicode.string)
        # get year(s)
        myear = re.search(r"\d+", line) #r"\d{4}" ?
        if myear:
            years.append(myear.group(0))
        else:
            year = None
        # get month(s)
        for month in MONTHS_MAPS.keys():
            if month in line.lower():
                months.append(MONTHS_MAPS[month])
    years = list(set(years))
    months = list(set(months))
    return years, months


def make_combinations(years, months):
    """Create every possible combination of months and years"""
    return [f"{yyyy}-{mm}" for yyyy, mm in itertools.product(years, months)]


def get_months_and_years(tree):
    """Get month(s) and year(s) a page refers to"""
    main_date = [elem for elem in tree.find_all(True, custom=True) if elem.attrs["custom"] == 'structure {type:main_date;}']
    years, months = build_list_of_years_and_months(main_date)
    return make_combinations(years, months)


def complete_wheniso_attrs(tei_tree, yyyy_mms):
    """Change values in wheniso attrs"""
    for date in tei_tree.find_all("date"):
        if "when-iso" in date.attrs.keys():
            new_values = [f"{yyyy_mm}-{date.attrs['when-iso']}" for yyyy_mm in yyyy_mms]
            if len(new_values) == 1:
                date.attrs["when-iso"] == new_values[0]
            elif len(new_values) > 1:
                date.attrs["when-iso"] = f"[{', '.join(new_values)}]"
    return tei_tree

#### 1.4.2 Contenu du TEI Header

#### 1.4.3 Constructions des unités logiques et des fichiers TEI

In [12]:
def build_new_row(xml_tree, row):
    new_row = xml_tree.new_tag("row")
    # 1. numero de répertoire
    cell = xml_tree.new_tag("cell", n=1, role="col1")
    for num_rep in row.entry_id:
        matching_tag = xml_tree.find(True, attrs={"xml:id": num_rep})
        cell.append(xml_tree.new_tag("lb", facs=f"#{num_rep}"))
        cell.append(NavigableString(matching_tag.line.text))
    new_row.append(cell)
    # 2. date de l'acte
    cell = xml_tree.new_tag("cell", n=2, role="col2")
    for date_act in row.date_of_act:
        matching_tag = xml_tree.find(True, attrs={"xml:id": date_act})
        cell.append(xml_tree.new_tag("lb", facs=f"#{date_act}"))
        cell_content = matching_tag.line.text.strip()
        if cell_content.isdigit:
            cell_date = xml_tree.new_tag("date", attrs={"when-iso":cell_content}) #TODO: mieux construire @when
            cell_date.append(NavigableString(cell_content))
            cell.append(cell_date)
        else:
            cell.append(NavigableString(cell_content))
    new_row.append(cell)
    # 3. type d'acte brevet
    cell = xml_tree.new_tag("cell", n=3, role="col3")
    for type_brevet in row.type_of_act["brevet"]:
        matching_tag = xml_tree.find(True, attrs={"xml:id": type_brevet})
        cell.append(xml_tree.new_tag("lb", facs=f"#{type_brevet}"))
        cell.append(NavigableString(matching_tag.line.text))
    new_row.append(cell)
    # 4. type d'acte minute
    cell = xml_tree.new_tag("cell", n=4, role="col4")
    for type_minute in row.type_of_act["minute"]:
        matching_tag = xml_tree.find(True, attrs={"xml:id": type_minute})
        cell.append(xml_tree.new_tag("lb", facs=f"#{type_minute}"))
        cell.append(NavigableString(matching_tag.line.text))
    new_row.append(cell)
    # 5. paragraphe central
    cell = xml_tree.new_tag("cell", n=5, role="col5")
    for mainp in row.main_paragraph:
        n = row.main_paragraph.index(mainp) + 1
        matching_tag = xml_tree.find(True, attrs={"xml:id": mainp})
        cell.append(xml_tree.new_tag("lb", n=n, facs=f"#{mainp}"))
        cell.append(NavigableString(matching_tag.line.text))
    new_row.append(cell)
    # 6. date enregistrement
    cell = xml_tree.new_tag("cell", n=6, role="col6")
    for date_reg in row.registration_relation["date"]:
        matching_tag = xml_tree.find(True, attrs={"xml:id": date_reg})
        cell.append(xml_tree.new_tag("lb", facs=f"#{date_reg}"))
        cell_content = matching_tag.line.text.strip()
        if cell_content.isdigit:
            cell_date = xml_tree.new_tag("date", attrs={"when-iso":cell_content}) #TODO: mieux construire @when
            cell_date.append(NavigableString(cell_content))
            cell.append(cell_date)
        else:
            cell.append(NavigableString(cell_content))
    new_row.append(cell)
    # 7. droit enregistrement
    cell = xml_tree.new_tag("cell", n=7, role="col7")
    for droits_reg in row.registration_relation["droits"]:
        matching_tag = xml_tree.find(True, attrs={"xml:id": droits_reg})
        cell.append(xml_tree.new_tag("lb", facs=f"#{droits_reg}"))
        cell.append(NavigableString(matching_tag.line.text))
    new_row.append(cell)
    # 8. misc
    cell = xml_tree.new_tag("cell", n=8, role="misc")
    for misc in row.misc:
        n = row.misc.index(misc) + 1
        matching_tag = xml_tree.find(True, attrs={"xml:id": misc})
        cell.append(xml_tree.new_tag("lb", n=n, facs=f"#{misc}"))
        cell.append(NavigableString(matching_tag.line.text))
    new_row.append(cell)
    return new_row


def update_tei_header(tei_tree):
    # 1. modification des balises "title" et "author"
    tei_tree.fileDesc.replace_with(BeautifulSoup(TEIHEADER["fileDesc"], "xml").fileDesc.extract())
    tei_tree.titleStmt.title.string = tei_file.split("/")[-1].replace(".xml", "")
    tei_tree.titleStmt.author.string = "Lectaurep"
    if len(tei_tree.find_all("encodingDesc")) == 1:
        tei_tree.encodingDesc.replace_with(BeautifulSoup(TEIHEADER["encodingDesc"], "xml").encodingDesc.extract())
    else:
        tei_tree.fileDesc.insert_after(BeautifulSoup(TEIHEADER["encodingDesc"], "xml").encodingDesc.extract())
    return tei_tree


def add_table_structure(tei_tree):
    # 1. construction des noeuds tempo_text (à renommer text) et body
    tei_tree.sourceDoc.insert_after(tei_tree.new_tag("tempo_text"))
    tei_tree.tempo_text.append(tei_tree.new_tag("body"))
    # 2. creation du tableau
    tei_tree.body.append(tei_tree.new_tag("div", type="main"))
    tei_tree.body.div.append(tei_tree.new_tag("table", rows=len(rows), cols=8))
    table_labels = tei_tree.new_tag("row", role="label", n=0)
    labels = ["Numéros du répertoire", "Dates des actes", "Actes en brevets", 
                        "Actes en minutes", "Noms, prénms et domiciles des parties ; indication, situations et prix des biens", 
                        "Date de l'enregistrement", "Droits de l'enregistrement", "Autres"]
    for label in labels:
        cell = tei_tree.new_tag("cell", role=f"label{labels.index(label) + 1}", n=labels.index(label) + 1)
        cell.append(NavigableString(label))
        table_labels.append(cell)
    tei_tree.body.div.table.append(table_labels)
    return tei_tree


def modify_tei_file(tei_file, rows, yyyy_mms):
    # 1. ouverture du fichier TEI
    xml_tree = open_and_parse_file(tei_file)
    # 2. modification du teiHeader
    xml_tree = update_tei_header(xml_tree)
    # 3. ajout de la structure table
    xml_tree = add_table_structure(xml_tree)
    # 4. ajout du contenu de table
    for row in rows:
        xml_tree.body.div.table.append(build_new_row(xml_tree, row))
    # 5. nettoyage des éléments temporaires
    text_node = xml_tree.find_all('tempo_text')
    if len(text_node) >= 1:
        text_node[0].name = "text"
    # 6. standardisation des dates dans les attributs when-iso
    control_dates(xml_tree)
    complete_wheniso_attrs(xml_tree, yyyy_mms)
    # 7. sauvegarde du fichier tei modifié
    tei4publisher_file = tei_file.replace("tei_output/", "tei4publisher/")
    save_file(tei4publisher_file, xml_tree.prettify().replace("&amp;amp;", "&amp;"))

## 2\. Application de la chaîne de transformation

### 2.1 Téléchargement des fichiers source

 `url` correspond à l'url de téléchargement des transcriptions générées dans eScriptorium (cf. prérequis)

In [20]:
url = "https://traces6.paris.inria.fr/media/users/4/export_test_vers_tei_publisher_pagexml_202106181347.zip"
filename = os.path.join("content/source", os.path.basename(url))
absfilename = os.path.abspath(filename)

!cd content/source && wget -N $url --no-check-certificate && unzip -u $absfilename && rm $filename
# Ajout du paramètre --no-check-certificate pour télécharger l'export eScriptorium depuis son API sans être authentifié

sources = [f for f in os.listdir("content/source/") if f.endswith(".xml")]

--2021-09-17 00:42:10--  https://traces6.paris.inria.fr/media/users/4/export_test_vers_tei_publisher_pagexml_202106181347.zip
Résolution de traces6.paris.inria.fr (traces6.paris.inria.fr)... 128.93.101.11
Connexion vers traces6.paris.inria.fr (traces6.paris.inria.fr)|128.93.101.11|:443... connecté.
AVERTISSEMENT : impossible de vérifier l'attribut traces6.paris.inria.fr du certificat, émis par «CN=GEANT OV RSA CA 4,O=GEANT Vereniging,C=NL» :
  Impossible de vérifier localement le certificat autorité de l'émetteur.
requête HTTP transmise, en attente de la réponse... 304 Not Modified
Le fichier «export_test_vers_tei_publisher_pagexml_202106181347.zip» n’a pas été modifié sur le serveur. Téléchargement sauté.

Archive:  /home/achague-vm/Documents/lepidemo/content/source/export_test_vers_tei_publisher_pagexml_202106181347.zip
rm: impossible de supprimer 'content/source/export_test_vers_tei_publisher_pagexml_202106181347.zip': Aucun fichier ou dossier de ce type


### 2.2 Application de la chaîne sur tous les fichiers XML PAGE téléchargés

In [23]:
for source in sources:
    # open xml page file
    source = os.path.join("content/source/", source)
    page_tree = open_and_parse_file(source)
    # get months and years combinations
    yyyy_mms = get_months_and_years(page_tree)
    # analyze page tree and create rows    
    rows = []
    for entry in vertical_slicing(page_tree):
        current_row = Row(page_tree, entry)
        rows.append(current_row)
    # generate tei file
    tei_file = source.replace(".xml", "-tei.xml").replace("/source/", "/tei_output/")
    !java -jar $path_to_saxon -xsl:$path_to_xsl -s:$source -o:$tei_file
    # modify tei file and save it
    modify_tei_file(tei_file, rows, yyyy_mms)

### 2.3 Génération d'une archive pour faciliter le téléchargement

Comme Google Colab ne permet pas de télécharger plusieurs fichiers à la fois, on zippe le contenu du dossier `tei4publisher/`

In [25]:
# zip the content of tei4publisher/
!zip -j content/tei4publisher.zip -r content/tei4publisher/

# zip the content of tei_output/
!zip -j content/tei_output.zip -r content/tei_output/

  adding: FRAN_0025_5094_L-1-tei.xml (deflated 72%)
  adding: FRAN_0025_0227_L-0-tei.xml (deflated 74%)
  adding: FRAN_0025_4648_L-1-tei.xml (deflated 74%)
  adding: FRAN_0025_3657_L-1-tei.xml (deflated 73%)
  adding: FRAN_0025_6067_L-1-tei.xml (deflated 74%)
  adding: FRAN_0025_3056_L-0-tei.xml (deflated 72%)
  adding: DAFANCH96_023MIC07633_L-0-tei.xml (deflated 66%)
  adding: FRAN_0025_1290_L-1-tei.xml (deflated 73%)
  adding: DAFANCH96_023MIC07645_L-0-tei.xml (deflated 68%)
  adding: FRAN_0025_5795_L-1-tei.xml (deflated 70%)
  adding: FRAN_0025_5094_L-1-tei.xml (deflated 72%)
  adding: FRAN_0025_0227_L-0-tei.xml (deflated 75%)
  adding: FRAN_0025_4648_L-1-tei.xml (deflated 73%)
  adding: FRAN_0025_3657_L-1-tei.xml (deflated 72%)
  adding: FRAN_0025_6067_L-1-tei.xml (deflated 72%)
  adding: FRAN_0025_3056_L-0-tei.xml (deflated 71%)
  adding: DAFANCH96_023MIC07633_L-0-tei.xml (deflated 65%)
  adding: FRAN_0025_1290_L-1-tei.xml (deflated 74%)
  adding: DAFANCH96_023MIC07645_L-0-tei.xml

---

## 3. Expérimentation : récupération des types d'actes à l'aide d'un référentiel et de recherche approximative

### 3.1 Installation et import des dépendances

On installe avec pip la librairie [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy).

In [26]:
!pip install fuzzywuzzy

On importe la librairie en question.

In [27]:
import json

from fuzzywuzzy import fuzz

/home/achague-vm/Documents/lepidemo/env/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### 3.2 Traitement du référentiel des types d'actes

On télécharge le référentiel depuis [Gitlab](https://gitlab.inria.fr/almanach/lectaurep/ner/-/blob/master/referentiels/referentiel_types_acte.json).

In [28]:
!wget -N https://gitlab.inria.fr/almanach/lectaurep/ner/-/raw/master/referentiels/referentiel_types_acte.json

--2021-09-17 00:50:48--  https://gitlab.inria.fr/almanach/lectaurep/ner/-/raw/master/referentiels/referentiel_types_acte.json
Résolution de gitlab.inria.fr (gitlab.inria.fr)... 128.93.193.23
Connexion vers gitlab.inria.fr (gitlab.inria.fr)|128.93.193.23|:443... connecté.
requête HTTP transmise, en attente de la réponse... 200 OK
Taille : 15428 (15K) [text/plain]
Enregistre : «referentiel_types_acte.json»

referentiel_types_a 100%[===================>]  15,07K  --.-KB/s    ds 0,001s  

En-tête de dernière modification manquant -- horodatage ignoré.
2021-09-17 00:50:48 (13,8 MB/s) - «referentiel_types_acte.json» enregistré [15428/15428]



On stocke le référentiel des types d'actes dans une variable.

In [29]:
#import json
with open("referentiel_types_acte.json", 'r', encoding='utf-8') as fh:
    referentiel_actes_json = json.load(fh)

## 3.3 Recherche floue des types d'actes dans la colonne 3 et 4 grâce au référentiel des types d'actes

In [30]:
tei_sources = [f for f in os.listdir("./content/tei4publisher/") if f.endswith(".xml")]

# On itère sur chaque fichier TEI
for source in tei_sources:
    print(f'\n TEI FILE : {source}')
    source = os.path.join("./content/tei4publisher/", source)
    xml_tree = open_and_parse_file(source)
    # On va chercher toutes les "cell" dans le fichier TEI
    cells = xml_tree.find_all('cell')

    # On itère dans les cellules pour trouver celles qui nous intéressent
    # Les cellules 3 et 4
    for cell in cells:
        if cell.has_attr('n'):
            if cell['n'] == "3" or cell['n'] == '4':
                # On vérifie que les cellules que l'on traite n'appartiennent pas à l'en-tête
                if not cell["role"] == "label3" or not cell["role"] == "label4":
                    text_cell = cell.text
                    # Si la cellule n'est pas vide, on la tokenize simplement avec split()
                    if len(text_cell) > 0:
                        tokenized_cell = text_cell.split()

                        # On itère à travers les tokens à l'aide des index pour pouvoir traiter les multiword expressions dans le futur
                        for idx in range(len(tokenized_cell)):
                            for type_acte in referentiel_actes_json:
                                # On calcule le taux de "fuzziness" avec fuzz.ratio entre le token et chaque entrée du référentiel
                                # Le token est normalisé avec lower()
                                # On récupère les tokens avec un taux supérieur ou égal à 91
                                # TODO: expérimenter avec le taux de fuzziness
                                if fuzz.ratio(tokenized_cell[idx].lower(), type_acte) >= 91:
                                    # Pour visualisation, on print le token et le type d'acte dans le référentiel qui ont un taux de fuzziness supérieur ou égal à 91
                                    print(tokenized_cell[idx], type_acte)
                                    # On imprime le taux de fuzziness
                                    print(fuzz.ratio(tokenized_cell[idx].lower(), type_acte))
                                    print('----')

                                # TODO: Traiter les multiword expressions
                                # TODO: Annoter les entités directement dans le fichier TEI


 TEI FILE : FRAN_0025_5094_L-1-tei.xml
Procuration (procuration)
92
----
Procuration procuratio
95
----
Procuration procuration
100
----
Procuration procuration 
96
----
d° d°
100
----
d° d°
100
----
Substitution substituion
96
----
Substitution substitution
100
----
Procuration (procuration)
92
----
Procuration procuratio
95
----
Procuration procuration
100
----
Procuration procuration 
96
----

 TEI FILE : FRAN_0025_0227_L-0-tei.xml
Quittance quittance
100
----
Quittance quittances
95
----
testament testament
100
----
Obligation obligation
100
----
Inventaire inventaire
100
----
Vente vente
100
----
testament testament
100
----
Mainlevée main levée
95
----
Mainlevée main-levée
95
----
Mainlevée mainlevée
100
----
Mainlevée mainlevées
95
----
Vente vente
100
----
notification notification
100
----
Mainlevée main levée
95
----
Mainlevée main-levée
95
----
Mainlevée mainlevée
100
----
Mainlevée mainlevées
95
----
Procuration (procuration)
92
----
Procuration procuratio
95
----
Procurat